In [61]:
import tensorflow
from tensorflow.keras.datasets import imdb#
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , Dense , SimpleRNN , Dropout
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [62]:
max_features = 10000
#vocab size

In [63]:
(X_train , y_train) , (X_test , y_test) = imdb.load_data(num_words = max_features)

In [ ]:
# 1 -> positive sentiment
# 0 -> negative sentiment

In [64]:
X_train[0] , y_train[0]

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

In [65]:
#decoding the review
word_dict = imdb.get_word_index()
def review_decoder(ohe_review):
  review=""
  for i in ohe_review:
    for k , v in word_dict.items():
      if v+3 == i:
        review = review + " " + k
  return review
review_decoder(X_train[0])

" this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert is an amazing actor and now the same being director father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also to the two little boy's that played the of norman and paul they were just brilliant children are often left out of the list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole

In [66]:
#prepadding
max_len=500
X_train = pad_sequences(X_train , maxlen = max_len)
X_test = pad_sequences(X_test , maxlen = max_len)

In [67]:
#making the RNN
model = Sequential()
model.add(Embedding(max_features , 128))
model.add(SimpleRNN(128 , activation='relu' , return_sequences=True))
model.add(SimpleRNN(128 , activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(1 , activation='sigmoid'))

In [68]:
early_stopper = EarlyStopping(monitor='val_loss' , patience= 5, restore_best_weights=True)

In [69]:
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])

In [71]:
model.fit(X_train , y_train , epochs=5 , validation_data=(X_test , y_test) , callbacks=[early_stopper])

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 416s 528ms/step - accuracy: 0.5886 - loss: 15420817.0000 - val_accuracy: 0.7317 - val_loss: 0.5428
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 427s 508ms/step - accuracy: 0.6283 - loss: 1842.4187 - val_accuracy: 0.6144 - val_loss: 0.6453
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 440s 506ms/step - accuracy: 0.5935 - loss: 1.0200 - val_accuracy: 0.6188 - val_loss: 0.6574
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 437s 500ms/step - accuracy: 0.6705 - loss: 0.6562 - val_accuracy: 0.6476 - val_loss: 0.6183
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 405s 518ms/step - accuracy: 0.7333 - loss: 0.5477 - val_accuracy: 0.7097 - val_loss: 0.5592


In [75]:
#function to convert text to a form that can be given to model
def review_encoder(text):
  word_dict = imdb.get_word_index()
  words = text.split()
  encoded = []
  for word in words:
    if word.lower() in word_dict:
      encoded.append(word_dict[word.lower()] + 3)
  return pad_sequences([encoded] , maxlen = max_len)[0]

In [76]:
def predict_sentiment(review):
  processed_input = review_encoder(review).reshape(1,-1)
  prediction = model.predict([processed_input])
  if prediction > 0.5:
    return 'positive'
  else:
    return 'negative'

In [77]:
#testing the model
review = "fantastic movie!! loved it"
sentiment = predict_sentiment(review)
sentiment

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


'negative'